In [79]:
import pandas as pd
import glob,gzip
import os
import mwparserfromhell
import re

#Reading input data
# languages = glob.glob('../common/data/wikipedia_cleaned_list_1120/clean*')
languages = glob.glob('../common/data/wikipedia_cleaned_list_1120/clean_related_list_en.csv')

pages = {}
idsToWikidata = {}
for lang in languages:
    langcode =lang[-6:-4]
    pages[langcode] = pd.read_csv(lang, names=["QID", "page_id", "language", "page_title", "category"])
    pages[langcode] = pages[langcode].drop([0])
    pages[langcode] = pages[langcode].set_index("QID")
    
    pages[langcode]['Wikidata'] = pages[langcode].index
    #Here we build a dictionary mapping page_ids to Wikidata Item

    idsToWikidata[langcode] = pages[langcode][['page_id','Wikidata']].set_index('page_id').to_dict()['Wikidata']
print(len(idsToWikidata['en']))

2275


In [81]:
def process_dump(dump,  path):
    global lang
    global pages
    global idsToWikidata
    global vertices
    global visited
    a = True
    for page in dump:
        if page.namespace == 0:
            if page.title in vertices and page.title not in visited:
                for revision in page:
                    visited.add(page.title)
                    try:
                        temps = re.findall(r'{{\s?Main\s?\|.*?}}', revision.text)
                        ts = [i.strip().split('#')[0] for j in temps for i in j.strip('{{}} ').split('|')[1:] if '=' not in i]
                        yield revision.page.title, ts
                    except:
                        print("exception at ", page.title)
                        print(revision.text)
                        pass

In [ ]:
import mwxml
import json
import csv
import time
date = "20201120"

start = time.time()
langs = ['en']
for lang in langs:
    paths = glob.glob('../common/data/20201120/%swiki-20201120-pages-meta-current.xml.bz2' % lang)
                      
    print(paths)
    print(lang)
#     vertices = {'COVID19'}
    vertices = {'COVID-19 pandemic', 'Severe acute respiratory syndrome coronavirus 2', 'Coronavirus disease 2019'}
#     vertices = {'Pandemia de COVID-19', 'SARS-CoV-2', 'COVID-19'}
    visited = set()
#     adjacency_dict = {'코로나19 범유행': []} #
    adjacency_dict = {'COVID-19 pandemic': [], 'Severe acute respiratory syndrome coronavirus 2': [], 'Coronavirus disease 2019': []} #
#     adjacency_dict = {'Pandemia de COVID-19': [], 'SARS-CoV-2': [], 'COVID-19': []} #
    
    cont = 0
    if cont != 0:
        tt = open('%s_from_3seed_step%d.json'%(lang, cont), 'r')
        adjacency_dict = json.load(tt)
        visited = set(adjacency_dict.keys())
        vertices = {y for x in adjacency_dict.values() for y in x}
        vertices.update(visited)
    print(adjacency_dict)
    print(vertices)
    print(visited)
    while True:    
        childs = set()
        with open("%s_from_3seed_step%d.json" % (lang, cont), 'w', encoding='utf-8') as f:
            f.write(json.dumps(adjacency_dict, ensure_ascii=False))
        for parent, child in mwxml.map(process_dump, paths, threads = 16): # what meaning for threads?
            childs.update(child)
            adjacency_dict[parent] = child #
            print(parent, child)
        if childs.issubset(visited):
            break
        vertices.update(childs)
        cont += 1
        print(cont, len(vertices))
        print(time.time()-start)
    
f.close()
        
        
print(time.time()-start)

['../common/data/20201120/enwiki-20201120-pages-meta-current.xml.bz2']
en
{'COVID-19 pandemic': [], 'Severe acute respiratory syndrome coronavirus 2': [], 'Coronavirus disease 2019': []}
{'COVID-19 pandemic', 'Severe acute respiratory syndrome coronavirus 2', 'Coronavirus disease 2019'}
set()
COVID-19 pandemic ['COVID-19 pandemic by country and territory', 'COVID-19 pandemic cases', 'COVID-19 pandemic deaths', 'COVID-19 pandemic death rates by country', 'Transmission of COVID-19', 'Symptoms of COVID-19', 'Severe acute respiratory syndrome coronavirus 2', 'COVID-19 testing', 'Social distancing measures related to the COVID-19 pandemic', 'Face masks during the COVID-19 pandemic', 'Hand washing', 'COVID-19 vaccine', 'Mitigation of COVID-19', 'Management of COVID-19', 'Timeline of the COVID-19 pandemic', 'Timeline of the COVID-19 pandemic in 2019', 'COVID-19 pandemic lockdowns', 'COVID-19 pandemic by country and territory', 'National responses to the COVID-19 pandemic', 'COVID-19 pandemic 

In [63]:
vertices
adjacency_dict

{'COVID-19 pandemic': [],
 'Severe acute respiratory syndrome coronavirus 2': [],
 'Coronavirus disease 2019': []}